In [91]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import gmean, ttest_ind
from statsmodels.stats.multitest import multipletests
import warnings
warnings.filterwarnings('ignore')

In [38]:
data = pd.read_excel("/Users/mortezaabyadeh/Desktop/Proteomic data.xlsx")

In [59]:
data.head()
data.tail(20)

,Gene Symbol,"Abundances (Normalized): F4: Sample, AKA_06","Abundances (Normalized): F5: Sample, AKA_06","Abundances (Normalized): F6: Sample, AKA_06","Abundances (Normalized): F1: Sample, WT_Control","Abundances (Normalized): F2: Sample, WT_Control","Abundances (Normalized): F3: Sample, WT_Control"
4671,ptr-18,NaN,NaN,NaN,NaN,NaN,NaN
4672,lys-1,NaN,NaN,57475.650357,113282.348392,132649.203516,153340.661068
4673,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4674,ife-4,NaN,NaN,NaN,NaN,NaN,NaN
4675,NaN,130845.015625,509218.897126,NaN,NaN,NaN,127779.538782
4676,ced-2,NaN,NaN,NaN,NaN,NaN,NaN
4677,NaN,94371.828125,113185.006355,72317.262198,81954.819600,NaN,NaN
4678,exc-7,50377.160156,34106.626538,52436.388677,NaN,NaN,NaN
4679,irg-5,125498.523438,NaN,123979.182593,NaN,NaN,NaN
4680,mboa-3,381437.437500,377590.362938,376434.592649,126817.725412,123991.744729,64251.944134


In [40]:
data.info()
data.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4691 entries, 0 to 4690
Data columns (total 7 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Gene Symbol                                      3315 non-null   object 
 1   Abundances (Normalized): F4: Sample, AKA_06      4028 non-null   float64
 2   Abundances (Normalized): F5: Sample, AKA_06      3928 non-null   float64
 3   Abundances (Normalized): F6: Sample, AKA_06      4036 non-null   float64
 4   Abundances (Normalized): F1: Sample, WT_Control  3693 non-null   float64
 5   Abundances (Normalized): F2: Sample, WT_Control  3759 non-null   float64
 6   Abundances (Normalized): F3: Sample, WT_Control  3724 non-null   float64
dtypes: float64(6), object(1)
memory usage: 256.7+ KB


Gene Symbol                                        1376
Abundances (Normalized): F4: Sample, AKA_06         663
Abundances (Normalized): F5: Sample, AKA_06         763
Abundances (Normalized): F6: Sample, AKA_06         655
Abundances (Normalized): F1: Sample, WT_Control     998
Abundances (Normalized): F2: Sample, WT_Control     932
Abundances (Normalized): F3: Sample, WT_Control     967
dtype: int64

In [41]:
data.shape

(4691, 7)

In [42]:
control_cols = ["Abundances (Normalized): F1: Sample, WT_Control", "Abundances (Normalized): F2: Sample, WT_Control", "Abundances (Normalized): F3: Sample, WT_Control"]
treatment_cols = ["Abundances (Normalized): F4: Sample, AKA_06", "Abundances (Normalized): F5: Sample, AKA_06", "Abundances (Normalized): F6: Sample, AKA_06"]

In [43]:
def filter_rows(row):
    control_nan_count = row[control_cols].isna().sum()
    treatment_nan_count = row[treatment_cols].isna().sum()
    return not ((control_nan_count > 2) or (treatment_nan_count > 2))

filtered_data = data[data.apply(filter_rows, axis=1)]

In [44]:
filtered_data.shape
filtered_data.isna().sum()

Gene Symbol                                        1089
Abundances (Normalized): F4: Sample, AKA_06         115
Abundances (Normalized): F5: Sample, AKA_06          68
Abundances (Normalized): F6: Sample, AKA_06         111
Abundances (Normalized): F1: Sample, WT_Control     192
Abundances (Normalized): F2: Sample, WT_Control     127
Abundances (Normalized): F3: Sample, WT_Control     160
dtype: int64

In [45]:
def impute_group(group):
    imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=10, random_state=0)
    imputed_group = imputer.fit_transform(group)
    return pd.DataFrame(imputed_group, columns=group.columns, index=group.index)

control_imputed = impute_group(filtered_data[control_cols])
filtered_data[control_cols] = control_imputed

treatment_imputed = impute_group(filtered_data[treatment_cols])
filtered_data[treatment_cols] = treatment_imputed

In [46]:
filtered_data.isna().sum()

Gene Symbol                                        1089
Abundances (Normalized): F4: Sample, AKA_06           0
Abundances (Normalized): F5: Sample, AKA_06           0
Abundances (Normalized): F6: Sample, AKA_06           0
Abundances (Normalized): F1: Sample, WT_Control       0
Abundances (Normalized): F2: Sample, WT_Control       0
Abundances (Normalized): F3: Sample, WT_Control       0
dtype: int64

In [60]:
filtered_data.head(10)
filtered_data.tail(20)

,Gene Symbol,"Abundances (Normalized): F4: Sample, AKA_06","Abundances (Normalized): F5: Sample, AKA_06","Abundances (Normalized): F6: Sample, AKA_06","Abundances (Normalized): F1: Sample, WT_Control","Abundances (Normalized): F2: Sample, WT_Control","Abundances (Normalized): F3: Sample, WT_Control"
4600,NaN,8.504684e+04,157305.333488,1.075624e+05,97787.247525,63693.930225,91198.736246
4602,tomm-7,5.673173e+04,13256.183313,4.794214e+04,102223.099795,28923.744392,35568.599715
4609,scl-3,1.099054e+05,128700.905070,1.069767e+05,164790.249538,135881.868543,171821.667550
4612,NaN,5.635306e+04,73444.382665,4.369366e+04,99316.796567,100330.040547,93800.945850
4613,rsd-6,9.797865e+04,134507.336372,1.039185e+05,56602.006139,50665.126152,45129.105570
4618,gpb-2,2.996053e+05,390747.096576,2.485449e+05,112687.634345,37928.485404,128702.049252
4627,col-40,9.946698e+05,714239.980716,1.160183e+06,276635.537413,329765.063645,350706.948528
4631,nspg-6,1.160176e+06,114603.967863,1.097886e+06,119934.490846,275511.428734,286932.712334
4632,NaN,6.908520e+04,106335.508373,7.040464e+04,196314.712827,264606.276887,153796.899360
4653,popl-7,6.636223e+04,71861.270827,5.384417e+04,99782.125293,74837.587079,101640.557341


In [48]:
filtered_data.shape

(3826, 7)

In [53]:
filtered_data.dropna(how='all', inplace=True)

In [54]:
filtered_data.shape

(3826, 7)

In [55]:
dat = filtered_data.dropna(subset=["Gene Symbol"])

In [56]:
dat.shape

(2737, 7)

In [58]:
dat.tail(10)

,Gene Symbol,"Abundances (Normalized): F4: Sample, AKA_06","Abundances (Normalized): F5: Sample, AKA_06","Abundances (Normalized): F6: Sample, AKA_06","Abundances (Normalized): F1: Sample, WT_Control","Abundances (Normalized): F2: Sample, WT_Control","Abundances (Normalized): F3: Sample, WT_Control"
4613,rsd-6,9.797865e+04,134507.336372,1.039185e+05,56602.006139,50665.126152,45129.105570
4618,gpb-2,2.996053e+05,390747.096576,2.485449e+05,112687.634345,37928.485404,128702.049252
4627,col-40,9.946698e+05,714239.980716,1.160183e+06,276635.537413,329765.063645,350706.948528
4631,nspg-6,1.160176e+06,114603.967863,1.097886e+06,119934.490846,275511.428734,286932.712334
4653,popl-7,6.636223e+04,71861.270827,5.384417e+04,99782.125293,74837.587079,101640.557341
4662,gpc-1,1.147924e+05,158384.933980,1.055855e+05,273852.249874,287943.397969,256204.072141
4667,famk-1,1.597203e+05,200727.198754,1.517732e+05,90779.006811,106179.627940,117292.029518
4672,lys-1,5.866804e+04,65582.185881,5.747565e+04,113282.348392,132649.203516,153340.661068
4680,mboa-3,3.814374e+05,377590.362938,3.764346e+05,126817.725412,123991.744729,64251.944134
4683,fabA,9.725413e+04,147098.206271,8.631682e+04,300008.034419,195250.000900,212964.554967


In [63]:
dat['Mean_AKA'] = gmean(dat[treatment_cols], axis=1)
dat['Mean_CTRL'] = gmean(dat[control_cols], axis=1)

In [72]:
dat["Fold Change"] = dat['Mean_AKA']/dat['Mean_CTRL']
dat['log2_Fold Change'] = np.log2(dat['Fold Change'])

In [87]:
dat.head(10)
dat.tail(10)

,Gene Symbol,"Abundances (Normalized): F4: Sample, AKA_06","Abundances (Normalized): F5: Sample, AKA_06","Abundances (Normalized): F6: Sample, AKA_06","Abundances (Normalized): F1: Sample, WT_Control","Abundances (Normalized): F2: Sample, WT_Control","Abundances (Normalized): F3: Sample, WT_Control",Mean_AKA,Mean_CTRL,Fold Change,log2_Fold Change
4613,rsd-6,9.797865e+04,134507.336372,1.039185e+05,56602.006139,50665.126152,45129.105570,111051.256354,50582.376077,2.195454,1.134519
4618,gpb-2,2.996053e+05,390747.096576,2.485449e+05,112687.634345,37928.485404,128702.049252,307574.318317,81936.185442,3.753828,1.908362
4627,col-40,9.946698e+05,714239.980716,1.160183e+06,276635.537413,329765.063645,350706.948528,937597.599142,317457.550786,2.953458,1.562405
4631,nspg-6,1.160176e+06,114603.967863,1.097886e+06,119934.490846,275511.428734,286932.712334,526534.528984,211651.455523,2.487743,1.314838
4653,popl-7,6.636223e+04,71861.270827,5.384417e+04,99782.125293,74837.587079,101640.557341,63560.138715,91217.854831,0.696795,-0.521194
4662,gpc-1,1.147924e+05,158384.933980,1.055855e+05,273852.249874,287943.397969,256204.072141,124282.638558,272355.712613,0.456325,-1.131867
4667,famk-1,1.597203e+05,200727.198754,1.517732e+05,90779.006811,106179.627940,117292.029518,169454.784272,104175.367785,1.626630,0.701886
4672,lys-1,5.866804e+04,65582.185881,5.747565e+04,113282.348392,132649.203516,153340.661068,60472.389204,132081.322003,0.457842,-1.127078
4680,mboa-3,3.814374e+05,377590.362938,3.764346e+05,126817.725412,123991.744729,64251.944134,378481.434693,100342.827494,3.771883,1.915285
4683,fabA,9.725413e+04,147098.206271,8.631682e+04,300008.034419,195250.000900,212964.554967,107284.493200,231922.963836,0.462587,-1.112204


In [76]:
dat[treatment_cols] = dat[treatment_cols].apply(pd.to_numeric, errors='coerce')
dat[control_cols] = dat[control_cols].apply(pd.to_numeric, errors='coerce')

In [89]:
from scipy.stats import ttest_ind

# Compute t-test statistics and p-values for each row
t_stat, p_value = ttest_ind(treatment_values, control_values, equal_var=False, axis=1)

# Assign the results to new columns in dat1
dat1["T-Stat"] = t_stat
dat1["P-Value"] = p_value

In [94]:
dat1.head()

,Gene Symbol,"Abundances (Normalized): F4: Sample, AKA_06","Abundances (Normalized): F5: Sample, AKA_06","Abundances (Normalized): F6: Sample, AKA_06","Abundances (Normalized): F1: Sample, WT_Control","Abundances (Normalized): F2: Sample, WT_Control","Abundances (Normalized): F3: Sample, WT_Control",Mean_AKA,Mean_CTRL,Fold Change,log2_Fold Change,T-Stat,P-Value,Adjusted_P_Value_Bonferroni
0,unc-54,2.198770e+09,2.403313e+09,2.119532e+09,2.053491e+09,1.964178e+09,1.994578e+09,2.237399e+09,2.003741e+09,1.116611,0.159126,2.671450,0.096707,1.000000
1,unc-15,7.118967e+08,7.818299e+08,6.823286e+08,1.397469e+09,1.394931e+09,1.337323e+09,7.241707e+08,1.376292e+09,0.526175,-0.926384,-18.375263,0.000137,0.373824
2,anc-1,3.937618e+08,5.092107e+08,3.950848e+08,9.184095e+08,8.974064e+08,9.151864e+08,4.294776e+08,9.102871e+08,0.471805,-1.083739,-12.304936,0.005273,1.000000
3,pdi-2,7.682669e+08,9.616308e+08,7.599919e+08,1.227115e+09,1.165712e+09,1.173886e+09,8.249795e+08,1.188596e+09,0.694079,-0.526828,-5.229931,0.024605,1.000000
4,act-1,2.255014e+09,2.002900e+09,2.408061e+09,1.315817e+09,1.515840e+09,1.475102e+09,2.215603e+09,1.432926e+09,1.546210,0.628736,5.914871,0.009682,1.000000


In [92]:
p_values = dat1["P-Value"]

adjusted_p_values = multipletests(p_values, method='bonferroni')[1]

dat1["Adjusted_P_Value_Bonferroni"] = adjusted_p_values

In [93]:
dat1.head()

,Gene Symbol,"Abundances (Normalized): F4: Sample, AKA_06","Abundances (Normalized): F5: Sample, AKA_06","Abundances (Normalized): F6: Sample, AKA_06","Abundances (Normalized): F1: Sample, WT_Control","Abundances (Normalized): F2: Sample, WT_Control","Abundances (Normalized): F3: Sample, WT_Control",Mean_AKA,Mean_CTRL,Fold Change,log2_Fold Change,T-Stat,P-Value,Adjusted_P_Value_Bonferroni
0,unc-54,2.198770e+09,2.403313e+09,2.119532e+09,2.053491e+09,1.964178e+09,1.994578e+09,2.237399e+09,2.003741e+09,1.116611,0.159126,2.671450,0.096707,1.000000
1,unc-15,7.118967e+08,7.818299e+08,6.823286e+08,1.397469e+09,1.394931e+09,1.337323e+09,7.241707e+08,1.376292e+09,0.526175,-0.926384,-18.375263,0.000137,0.373824
2,anc-1,3.937618e+08,5.092107e+08,3.950848e+08,9.184095e+08,8.974064e+08,9.151864e+08,4.294776e+08,9.102871e+08,0.471805,-1.083739,-12.304936,0.005273,1.000000
3,pdi-2,7.682669e+08,9.616308e+08,7.599919e+08,1.227115e+09,1.165712e+09,1.173886e+09,8.249795e+08,1.188596e+09,0.694079,-0.526828,-5.229931,0.024605,1.000000
4,act-1,2.255014e+09,2.002900e+09,2.408061e+09,1.315817e+09,1.515840e+09,1.475102e+09,2.215603e+09,1.432926e+09,1.546210,0.628736,5.914871,0.009682,1.000000


In [95]:
p_values = dat1["P-Value"]

adjusted_p_values_bh = multipletests(p_values, method='fdr_bh')[1]

dat1["Adjusted_P_Value_BH"] = adjusted_p_values_bh

In [97]:
dat1.head()
sorted_dat1 = dat1.sort_values(by="P-Value")
sorted_dat1.head()

,Gene Symbol,"Abundances (Normalized): F4: Sample, AKA_06","Abundances (Normalized): F5: Sample, AKA_06","Abundances (Normalized): F6: Sample, AKA_06","Abundances (Normalized): F1: Sample, WT_Control","Abundances (Normalized): F2: Sample, WT_Control","Abundances (Normalized): F3: Sample, WT_Control",Mean_AKA,Mean_CTRL,Fold Change,log2_Fold Change,T-Stat,P-Value,Adjusted_P_Value_Bonferroni,Adjusted_P_Value_BH
523,kars-1,1.760476e+07,1.727160e+07,1.776617e+07,5.035147e+06,5.269753e+06,4.889098e+06,1.754630e+07,5.062251e+06,3.466105,1.793316,68.205315,6.383468e-07,0.001747,0.000922
230,ucr-1,8.039260e+07,8.114768e+07,8.250667e+07,1.680544e+07,1.949516e+07,1.833806e+07,8.134429e+07,1.817928e+07,4.474561,2.161746,63.474377,6.740519e-07,0.001845,0.000922
237,pat-2,1.845986e+07,1.822675e+07,1.822662e+07,1.297346e+07,1.320778e+07,1.290627e+07,1.830408e+07,1.302853e+07,1.404923,0.490491,43.971186,2.099974e-06,0.005748,0.001000
1989,npp-13,7.357411e+05,7.628505e+05,7.686747e+05,1.379339e+05,1.713631e+05,1.662669e+05,7.556181e+05,1.578089e+05,4.788186,2.259479,41.106060,2.106414e-06,0.005765,0.001000
716,idhg-1,1.243450e+07,1.284608e+07,1.230758e+07,3.340597e+06,3.808022e+06,3.724014e+06,1.252729e+07,3.618359e+06,3.462147,1.791667,41.027136,2.458479e-06,0.006729,0.001000


In [98]:
sorted_dat1.to_excel("/Users/mortezaabyadeh/Desktop/Cleaned_Proteomic_data.xlsx", index=False)

In [103]:
dat2 = dat1[dat1["P-Value"] < 0.05]
dat2.head()

,Gene Symbol,"Abundances (Normalized): F4: Sample, AKA_06","Abundances (Normalized): F5: Sample, AKA_06","Abundances (Normalized): F6: Sample, AKA_06","Abundances (Normalized): F1: Sample, WT_Control","Abundances (Normalized): F2: Sample, WT_Control","Abundances (Normalized): F3: Sample, WT_Control",Mean_AKA,Mean_CTRL,Fold Change,log2_Fold Change,T-Stat,P-Value,Adjusted_P_Value_Bonferroni,Adjusted_P_Value_BH
1,unc-15,7.118967e+08,7.818299e+08,6.823286e+08,1.397469e+09,1.394931e+09,1.337323e+09,7.241707e+08,1.376292e+09,0.526175,-0.926384,-18.375263,0.000137,0.373824,0.004734
2,anc-1,3.937618e+08,5.092107e+08,3.950848e+08,9.184095e+08,8.974064e+08,9.151864e+08,4.294776e+08,9.102871e+08,0.471805,-1.083739,-12.304936,0.005273,1.000000,0.024257
3,pdi-2,7.682669e+08,9.616308e+08,7.599919e+08,1.227115e+09,1.165712e+09,1.173886e+09,8.249795e+08,1.188596e+09,0.694079,-0.526828,-5.229931,0.024605,1.000000,0.053755
4,act-1,2.255014e+09,2.002900e+09,2.408061e+09,1.315817e+09,1.515840e+09,1.475102e+09,2.215603e+09,1.432926e+09,1.546210,0.628736,5.914871,0.009682,1.000000,0.031929
6,ACTB,1.498700e+07,9.632594e+06,1.778686e+07,3.257887e+06,4.646444e+06,5.082788e+06,1.369365e+07,4.253237e+06,3.219582,1.686873,3.995155,0.048435,1.000000,0.083796


In [105]:
dat2_sorted = dat2.sort_values(by="Fold Change")
dat2_sorted.head()

,Gene Symbol,"Abundances (Normalized): F4: Sample, AKA_06","Abundances (Normalized): F5: Sample, AKA_06","Abundances (Normalized): F6: Sample, AKA_06","Abundances (Normalized): F1: Sample, WT_Control","Abundances (Normalized): F2: Sample, WT_Control","Abundances (Normalized): F3: Sample, WT_Control",Mean_AKA,Mean_CTRL,Fold Change,log2_Fold Change,T-Stat,P-Value,Adjusted_P_Value_Bonferroni,Adjusted_P_Value_BH
2597,test-1,490968.593750,82667.394900,647735.761407,6.199175e+06,4.718704e+06,4.884000e+06,297345.894393,5.227701e+06,0.056879,-4.135963,-9.764496,0.004647,1.000000,0.022553
1861,ALB,112562.265625,275441.202657,71386.903008,1.511328e+06,1.408729e+06,1.529397e+06,130320.706096,1.482184e+06,0.087925,-3.507587,-18.281037,0.000203,0.556438,0.005402
3734,spp-2,301063.937500,371621.451632,335348.379061,3.435094e+06,2.415176e+06,3.958351e+06,334774.143867,3.202338e+06,0.104541,-3.257866,-6.467794,0.022849,1.000000,0.051175
2860,ccdc-12,94768.484375,188099.734571,124017.920708,9.479811e+05,1.093388e+06,9.109420e+05,130270.332352,9.810435e+05,0.132788,-2.912809,-13.657235,0.000967,1.000000,0.011158
2326,cspE,133889.390625,223234.808649,172731.442805,1.217230e+06,1.246049e+06,1.180186e+06,172832.930326,1.214188e+06,0.142344,-2.812542,-32.301651,0.000012,0.032693,0.001487


In [109]:
with pd.ExcelWriter('/Users/mortezaabyadeh/Desktop/Cleaned_Proteome.xlsx', engine='xlsxwriter') as writer:
    dat2_sorted.to_excel(writer, sheet_name='All Data', index=False)

    low_abundance = dat2_sorted[dat2_sorted["log2_Fold Change"] < 0]
    high_abundance = dat2_sorted[dat2_sorted["log2_Fold Change"] > 0]

    low_abundance.to_excel(writer, sheet_name='Low Abundance', index=False)
    high_abundance.to_excel(writer, sheet_name='High Abundance', index=False)

    writer.save()